# Cohort Analysis

Group customers - enhance product offering and marketing strategy

Group by recency,freuqnecy values and other purchasing behaviorsb

Cohort analysis is a descriptive analysis - groups customers into mutually exclsuive cohort

3 types of cohort
1. time cohort (daily, weekly, monthly)
2. behavior cohort (group by type of product,service)
3. size cohort (by spending)

## Time Cohort Analysis

data: pivot table - rows: time, column: k-th cohort
values - total number of customers 

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('/Users/jihunlee/Downloads/Online Retail.xlsx')

In [4]:
df.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


### Create Pivot Table

In [ ]:
def get_month(x): 
    return dt.datetime(x.year, x.month, 1)

In [ ]:
df['InvoiceMonth'] = df['InvoiceDate'].apply(get_month)

In [ ]:
grouping = online.groupby('CustomerID')['InvoiceMonth']

In [ ]:
df['CohortMonth'] = grouping.transform('min')

In [5]:
def get_data_int(df, column):
    year = df[column].dt.year
    month = df[column].dt.month
    day = df[column].dt.day
    return year, month, day

In [ ]:
invoice_year, invoice_month,_ = get_date_int(df, 'InvoiceMonth')
cohort_year, cohort_month, _ = get_date_int(df, 'CohortMonth')

In [ ]:
years_diff = invoice_year - cohort_year
months_diff = invoice_month - cohort_month

In [ ]:
df['CohortIndex'] = years_diff*12 + month_diff + 1

In [ ]:
grouping = df.groupby(['CohortMonth','CohortIndex'])

In [ ]:
# Count the number of unique values per customer ID
cohort_data = grouping['CustomerID'].apply(pd.Series.nunique)

In [ ]:
cohort_data = cohort_data.reset_index()

In [ ]:
# Create a pivot 
cohort_counts = cohort_data.pivot(index='CohortMonth',
                                 columns = 'CohortIndex',
                                 values = 'CustomerID')

## Create Metrics

1. Retention Rate <br>

Retention = Percentage of active customers out of total customers

In [ ]:
# Select the first column and store it to cohort_sizes
cohort_sizes = ochort_counts.iloc[:,0]

In [ ]:
# Divide the cohort count by cohort sizes along the rows
retention = cohort_counts.divide(cohort_sizes, axis=0)

In [ ]:
retention.round(3)*100

2. Average Quantity Purchased

In [ ]:
#  Create a groupby object and pass the monthly cohort and cohort index as a list
grouping = online.groupby(['CohortMonth','CohortIndex'])

In [ ]:
# Calculate the average of the unit price column
cohort_data = grouping['Quantity'].mean()

In [ ]:
# Reset the index of cohort_data
cohort_Data = cohort_data.reset_index()

In [ ]:
average_quantity = cohort_data.pivot(index='CohortMonth',
                                    columns='CohortIndex',
                                    values='Quantity')

In [ ]:
average_quantity.round(1)

## Visualize It

- heatmap is the easiest way to visualize cohort analysis
- includes both actual metric values and color coding to see the differences in the numbers visually

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(10,8)) # initialize a figure of width and height in inches
plt.title('Retention Rates')
sns.heatmap(data=retention,
           annot=True, # print the numbers
           fmt='.0%', # format by percentage with one decimal value
           vmin=0.0, # anchor the colormap and make sue the outliers dont impact the visualization
           vmax=0.5,
           cmap='BuGn') # blue green color palette
plt.show()

# Recency, Frequency, and Monetary Value Analysis

Three customer behavior metrics:
    1. Recency: how recent was the last purchase
    2. Frequency: how many purchases customer had
    3. Monetary value: how much customer has spent in the last 12 months

Group RFM vaues into:
    1. Percenties e.g. quantiles
    2. Pareto 80/20 split
    3. Custom - based on business knowledge

Percentiles:
    1. Sort customers based on metric
    2. Break customers into a predefined number of groups of equal size
    3. Assign a label to each group

In [ ]:
# Create a spend quartile with 4 groups - a range between 1 and 5
spend_quartiles = pd.qcut(data['Spend'], q=4, labels=range(1,5)) # integer names

In [ ]:
# Assign the quartile values to the Spend_Quartile column in data
data['Spend_Quartile'] = spend_quartiles
data.sort_values('Spend')

Assigning Labels:
    - Highest score to the best metric - best is not always highest e.g. recency

In [ ]:
# Store labels from 4 to 1 in a decreasing order
r_labels = list(range(4,0,-1))

In [ ]:
# Create a spend quartile with 4 groups and pass the previously created labels 
recency_quartiles = pd.qcut(data['Recency_Days'], q=4, labels=r_labels)

In [ ]:
# Assign the quartile values to the Recency_Quartile column in `data`
data['Recency_Quartile'] = recency_quartiles
# sort recency values from lowest to highest
data.sort_values('Recency_Days')

## Calculating RFM Metrics

Recency - days since last customer transaction; we want customer to be recent <br>

Frequency - number of transactions in the last 12 months <br>

Monetary Value - total spend in the last 12 months

In [ ]:
print('Min:{}; Max:{}'.format(min(online.InvoiceDate),
                             max(online.InvoiceDate)))

In [ ]:
# Create a hypothetical snapshot_day data as if we are doing this analysis recently
snapshot_date = max(online.InvoiceDate) + datetime.timedelta(days=1) # period of one day we add to day

In [ ]:
# Aggregate data on a customer level
# Calculate Recency, Frequency and Monetary value for each customer
datamart = online.groupby(['CustomerID']).agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'count',
    'TotalSu': 'sum'})

In [ ]:
# Rename columns for easier Interpretation
datamart.rename(columns={'InvoiceDate': 'Recency',
                        'InvoiceNo': 'Frequency',
                        'TotalSum': 'MonetaryValue'}, inplace=True)

## Recency Quartile

In [ ]:
# Create labels for Recency, Frequency and Monetary Value
r_labels = range(4,0,-1)
f_labels = range(1,5)
m_labels = range(1,5)

# Assign these labels to three equal percentile groups 
r_quartiles = pd.qcut(datamart['Recency'], q=4, labels=r_labels)
f_quartiles = pd.qcut(datamart['Frequency'], q=4, labels=f_labels)
m_quartiles = pd.qcut(datamart['MonetaryValue'], q=4, labels=m_labels)

# Create new columns R and F
datamart = datamart.assign(F = f_quartiles.values)
datamart = datamart.assign(M = m_quartiles.values)
datamart = datamart.assign(R = r_quartiles.values)

In [ ]:
# Concatenate RFM quartile values
def join_rfm(x):
    return str(x['R']) + str(x['F']) + str(x['M'])
datamart['RFM_Segment'] = datamart.apply(join_rfm, axis=1)
# Calculate RFM_Score based on the sum of R, F, and M column values.
datamart['RFM_Score'] = datamart[['R','F','M']].sum(axis=1)

## Analyzing RFM Segments

investigate size of segments

In [ ]:
datamart.groupby('RFM_Segment').size().sort_values(ascending=False)[:10]

Simple selection of segments based on RFM Scores

In [ ]:
# select bottom RFM segment '111' and view top 5 rows
datamart[datamart['RFM_Segment']=='111'][:5]

In [ ]:
# summary metrics per RFM score
datamart.groupby('RFM_Score').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'MonetaryValue': ['mean', 'conut']
}).round(1)

Use RFM score to group customers into Gold Silver and Bronze segments

In [ ]:
def segment_me(df):
    if df['RFM_Score'] >= 0:
        return 'Gold'
    elif (df['RFM_Score'] >= 5) and (df['RFM_Score'] < 9):
        return 'Silver'
    else: 
        return 'Bronze'

In [ ]:
datamart['General Segment'] = datamart.apply(segment_me, axis=1)

In [ ]:
# Calculate average values for each general segment, and return a size of each segment 
datamart.groupby('General_Segment').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'MonetaryValue': ['mean','count'] # Return the size of each segment
}).round(1)